# Review Flow AI
Uma empresa de e-commerce recebe centenas de reviews diariamente e precisa:

- Analisar sentimentos e problemas reportados
- Categorizar feedback por tipo (qualidade, entrega, atendimento)
- Gerar respostas personalizadas para reviews negativos
- Criar relatórios para diferentes departamentos
- Escalar problemas críticos para supervisores


In [2]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict, Optional
import os
import asyncio

In [3]:
load_dotenv(override=True)

True

## Agent 1: Review Analyzer 

Agente responsável por analisar reviews de clientes, classificar sentimentos, identificar categorias de problemas e detectar urgência. Deve extrair informações estruturadas do texto livre e preparar dados para os próximos agentes.

In [5]:
review_analyzer_prompt = """
You are a Review Analyzer agent for an e-commerce company. Your job is to analyze customer reviews and extract structured information.

INPUT FORMAT:
- Review text (string)
- Review metadata (JSON):
{ 
  "id": "string", 
  "product_name": "string",
  "product_price": "number", 
  "purchase_date": "string", 
  "customer_id": "string", 
  "customer_name": "string" 
}

YOUR TASKS:
1. Analyze the sentiment and assign a score from 1-10 (1=very negative, 10=very positive)
2. Classify sentiment as: Positive, Neutral, or Negative
3. Identify problem categories (can be multiple): Product, Delivery, Customer_Service, Price
4. Detect urgency level: Low, Medium, or High (based on keywords like "urgent", "immediately", "terrible", "lawsuit", etc.)
5. Extract key issues mentioned in the review

OUTPUT FORMAT (JSON only):
{
  "sentiment": "Positive|Neutral|Negative",
  "sentiment_score": 1-10,
  "categories": ["Product", "Delivery", "Customer_Service", "Price"],
  "urgency": "Low|Medium|High",
  "key_issues": ["list of specific problems mentioned"],
  "customer_id": "string",
  "customer_name": "string",
  "product_name": "string"
}

Rules:
- Be objective and accurate in sentiment classification
- Identify ALL relevant categories, not just one
- Urgency High: mentions of legal action, health risks, or extreme dissatisfaction
- Urgency Medium: significant problems affecting product use
- Urgency Low: minor complaints or suggestions
- Keep key_issues concise (max 3-5 items)
"""

review_analyzer_agent = Agent(
    name="review_analyzer",
    instructions=review_analyzer_prompt, 
    model="gpt-4o-mini")

review_analyzer_tool = review_analyzer_agent.as_tool(
    tool_name="Review Analyzer",
    tool_description="Analyzes customer reviews and extracts structured information including sentiment, categories, urgency, and key issues."
)

## Agent 2: Response Generator
Agente responsável por gerar respostas personalizadas e empáticas para reviews negativos. Deve adaptar o tom baseado na severidade do problema e oferecer soluções específicas, evitando respostas genéricas.

In [6]:
response_generator_prompt = """
You are a Response Generator agent for an e-commerce customer service team. Your job is to create personalized, empathetic responses to negative customer reviews.

INPUT FORMAT:
- Analysis from Review Analyzer (JSON with sentiment, categories, urgency, key_issues)
- Original review text
- Customer metadata (JSON): 
{ 
  "id": "string", 
  "product_name": "string",
  "product_price": "number", 
  "purchase_date": "string", 
  "customer_id": "string", 
  "customer_name": "string" 
}

YOUR TASKS:
1. Generate a personalized response that addresses the customer by name
2. Acknowledge specific issues mentioned in the review
3. Show genuine empathy and understanding
4. Offer concrete solutions or compensation based on problem severity
5. Adapt tone based on urgency level

TONE GUIDELINES:
- High urgency: Very apologetic, immediate action focus, generous compensation offer
- Medium urgency: Apologetic, solution-oriented, reasonable compensation
- Low urgency: Friendly, helpful, focus on improvement

OUTPUT FORMAT (JSON):
{
  "response_text": "string (the complete response ready to publish)",
  "compensation_offered": "string or null (e.g., '20% discount', 'free shipping', 'full refund')",
  "tone_used": "High_urgency|Medium_urgency|Low_urgency"
}

Rules:
- Always use customer's name
- Never use generic templates like "We're sorry to hear that"
- Address EACH key issue specifically
- Keep responses between 3-5 sentences
- Be human and conversational, not robotic
- Offer compensation for Medium/High urgency issues
- End with a forward-looking statement
"""

response_generator_agent = Agent(
    name="response_generator", 
    instructions=response_generator_prompt, 
    model="gpt-4o-mini")

response_generator_tool = response_generator_agent.as_tool(
    tool_name="Response Generator",
    tool_description="Generates personalized, empathetic responses to negative customer reviews based on analysis."
)

## Agent 3: Escalation Manager
Agente responsável por identificar reviews que necessitam intervenção humana, categorizar o tipo de escalação necessária e gerar tickets estruturados com contexto, ações recomendadas e prioridades.

In [7]:
escalation_manager_prompt = """
You are an Escalation Manager agent. Your job is to identify critical reviews that require human intervention and create structured escalation tickets.

INPUT FORMAT:
- Analysis from Review Analyzer (JSON)
- Original review text
- Customer history summary (string): Use tool get_customer_history to retrieve previous purchases, past complaints, customer lifetime value

YOUR TASKS:
1. Determine if escalation is needed (only for High urgency OR specific trigger words)
2. Categorize escalation type: Technical, Commercial, or Legal
3. Create executive summary with context
4. Recommend specific actions
5. Assign priority (P1-Critical, P2-High, P3-Medium)
6. Suggest responsible department
7. Propose resolution timeline

ESCALATION TRIGGERS:
- Legal: mentions of lawsuit, lawyer, legal action, authorities
- Technical: product defects, safety issues, malfunctions
- Commercial: refund disputes, pricing errors, billing problems

OUTPUT FORMAT (JSON):
{
  "escalation_needed": true|false,
  "escalation_type": "Technical|Commercial|Legal|null",
  "priority": "P1|P2|P3|null",
  "department": "string (e.g., 'Product Quality Team', 'Legal Department')",
  "executive_summary": "string (2-3 sentences with context)",
  "recommended_actions": ["action1", "action2", "action3"],
  "suggested_timeline": "string (e.g., '24 hours', '3 business days')",
  "customer_value": "string (e.g., 'High-value customer', 'First-time buyer')"
}

Rules:
- Only escalate truly critical issues (High urgency + serious problems)
- P1 priority: legal threats, safety issues, viral potential
- P2 priority: significant product failures, high-value customers
- P3 priority: complex issues needing expert attention
- Keep executive summary clear and actionable
- Recommend 2-4 specific actions
- Consider customer history in priority assignment
"""

escalation_manager_agent = Agent(
    name="escalation_manager", 
    instructions=escalation_manager_prompt, 
    model="gpt-4o-mini")

escalation_manager_tool = escalation_manager_agent.as_tool(
    tool_name="Escalation Manager",
    tool_description="Identifies critical reviews for human intervention and creates structured escalation tickets."
)


In [9]:
@function_tool
def get_customer_history(customer_id: str) -> str:
    """
    Mock function to retrieve customer information based on customer ID.
    
    Args:
        customer_id (str): The unique identifier for the customer
        
    Returns:
        dict: Customer information including purchase history and profile data
        None: If customer not found
    """
    
    # Mock database of customers
    mock_customers = {
        "CUST-12345": {
            "customer_id": "CUST-12345",
            "customer_name": "João Silva",
            "email": "joao.silva@email.com",
            "phone": "+55 11 98765-4321",
            "registration_date": "2023-05-10",
            "customer_tier": "Gold",
            "total_purchases": 15,
            "total_spent": 4500.00,
            "lifetime_value": "High",
            "previous_complaints": 1,
            "complaint_history": [
                {
                    "date": "2024-08-15",
                    "issue": "Atraso na entrega",
                    "resolution": "Compensação com desconto de 15%",
                    "status": "Resolvido"
                }
            ],
            "recent_purchases": [
                {
                    "order_id": "ORD-789",
                    "product": "Smartphone XYZ",
                    "date": "2025-10-15",
                    "value": 1200.00,
                    "status": "Entregue"
                },
                {
                    "order_id": "ORD-756",
                    "product": "Fone Bluetooth ABC",
                    "date": "2025-09-20",
                    "value": 350.00,
                    "status": "Entregue"
                }
            ],
            "preferred_payment": "Cartão de Crédito",
            "average_order_value": 300.00
        },
        
        "CUST-67890": {
            "customer_id": "CUST-67890",
            "customer_name": "Maria Santos",
            "email": "maria.santos@email.com",
            "phone": "+55 21 91234-5678",
            "registration_date": "2025-01-15",
            "customer_tier": "Silver",
            "total_purchases": 3,
            "total_spent": 890.00,
            "lifetime_value": "Medium",
            "previous_complaints": 0,
            "complaint_history": [],
            "recent_purchases": [
                {
                    "order_id": "ORD-801",
                    "product": "Notebook Pro 15",
                    "date": "2025-10-20",
                    "value": 450.00,
                    "status": "Em trânsito"
                },
                {
                    "order_id": "ORD-745",
                    "product": "Mouse Wireless",
                    "date": "2025-08-10",
                    "value": 120.00,
                    "status": "Entregue"
                }
            ],
            "preferred_payment": "PIX",
            "average_order_value": 296.67
        },
        
        "CUST-11111": {
            "customer_id": "CUST-11111",
            "customer_name": "Carlos Oliveira",
            "email": "carlos.oliveira@email.com",
            "phone": "+55 31 99876-5432",
            "registration_date": "2024-11-20",
            "customer_tier": "Platinum",
            "total_purchases": 28,
            "total_spent": 12500.00,
            "lifetime_value": "Very High",
            "previous_complaints": 2,
            "complaint_history": [
                {
                    "date": "2025-06-10",
                    "issue": "Produto com defeito",
                    "resolution": "Troca imediata + desconto na próxima compra",
                    "status": "Resolvido"
                },
                {
                    "date": "2025-03-05",
                    "issue": "Atendimento inadequado",
                    "resolution": "Pedido de desculpas + cupom de R$100",
                    "status": "Resolvido"
                }
            ],
            "recent_purchases": [
                {
                    "order_id": "ORD-888",
                    "product": "Smart TV 55\"",
                    "date": "2025-10-25",
                    "value": 2800.00,
                    "status": "Processando"
                },
                {
                    "order_id": "ORD-850",
                    "product": "Soundbar Premium",
                    "date": "2025-09-30",
                    "value": 980.00,
                    "status": "Entregue"
                }
            ],
            "preferred_payment": "Cartão de Crédito",
            "average_order_value": 446.43
        },
        
        "CUST-22222": {
            "customer_id": "CUST-22222",
            "customer_name": "Ana Paula Costa",
            "email": "ana.costa@email.com",
            "phone": "+55 85 98123-4567",
            "registration_date": "2025-10-01",
            "customer_tier": "Bronze",
            "total_purchases": 1,
            "total_spent": 89.90,
            "lifetime_value": "Low",
            "previous_complaints": 0,
            "complaint_history": [],
            "recent_purchases": [
                {
                    "order_id": "ORD-900",
                    "product": "Capa para Celular",
                    "date": "2025-10-28",
                    "value": 89.90,
                    "status": "Entregue"
                }
            ],
            "preferred_payment": "Boleto",
            "average_order_value": 89.90
        },
        
        "CUST-33333": {
            "customer_id": "CUST-33333",
            "customer_name": "Pedro Henrique Lima",
            "email": "pedro.lima@email.com",
            "phone": "+55 41 97654-3210",
            "registration_date": "2024-03-22",
            "customer_tier": "Gold",
            "total_purchases": 12,
            "total_spent": 5600.00,
            "lifetime_value": "High",
            "previous_complaints": 3,
            "complaint_history": [
                {
                    "date": "2025-09-01",
                    "issue": "Produto errado enviado",
                    "resolution": "Troca + frete grátis",
                    "status": "Resolvido"
                },
                {
                    "date": "2025-05-15",
                    "issue": "Cobrança duplicada",
                    "resolution": "Estorno imediato",
                    "status": "Resolvido"
                },
                {
                    "date": "2024-12-20",
                    "issue": "Atraso de 10 dias",
                    "resolution": "Desconto de 20% aplicado",
                    "status": "Resolvido"
                }
            ],
            "recent_purchases": [
                {
                    "order_id": "ORD-920",
                    "product": "Teclado Mecânico RGB",
                    "date": "2025-10-30",
                    "value": 450.00,
                    "status": "Em separação"
                },
                {
                    "order_id": "ORD-890",
                    "product": "Webcam Full HD",
                    "date": "2025-10-05",
                    "value": 280.00,
                    "status": "Entregue"
                }
            ],
            "preferred_payment": "Cartão de Débito",
            "average_order_value": 466.67
        }
    }
    
    # Return customer info or None if not found
    customer = mock_customers.get(customer_id)
    
    if customer:
        return customer
    else:
        return None

In [10]:
@function_tool
def get_product_info(product_id: str) -> Optional[Dict]:
    """
    Mock function to retrieve product information based on product ID.
    
    Args:
        product_id (str): The unique identifier for the product
        
    Returns:
        dict: Product information including details, pricing, stock, and common issues
        None: If product not found
    """
    
    # Mock database of products
    mock_products = {
        "PROD-001": {
            "product_id": "PROD-001",
            "product_name": "Smartphone XYZ Pro",
            "category": "Eletrônicos",
            "subcategory": "Smartphones",
            "brand": "TechBrand",
            "price": 1200.00,
            "original_price": 1499.00,
            "discount_percentage": 20,
            "stock_status": "Em estoque",
            "stock_quantity": 45,
            "average_rating": 4.2,
            "total_reviews": 328,
            "description": "Smartphone top de linha com câmera de 108MP, 5G, tela AMOLED 6.7\"",
            "warranty_months": 12,
            "warranty_type": "Garantia do fabricante",
            "shipping_weight_kg": 0.5,
            "dimensions": "16cm x 8cm x 0.9cm",
            "common_issues": [
                {
                    "issue": "Tela quebrada no transporte",
                    "frequency": "Média",
                    "resolution": "Troca imediata + reforço na embalagem"
                },
                {
                    "issue": "Bateria com desempenho abaixo do esperado",
                    "frequency": "Baixa",
                    "resolution": "Atualização de firmware ou troca"
                }
            ],
            "return_policy_days": 30,
            "supplier": "TechBrand Brasil",
            "manufacturing_origin": "China",
            "release_date": "2025-01-15"
        },
        
        "PROD-002": {
            "product_id": "PROD-002",
            "product_name": "Notebook Pro 15",
            "category": "Eletrônicos",
            "subcategory": "Notebooks",
            "brand": "CompuMax",
            "price": 3500.00,
            "original_price": 4200.00,
            "discount_percentage": 17,
            "stock_status": "Estoque baixo",
            "stock_quantity": 8,
            "average_rating": 4.7,
            "total_reviews": 156,
            "description": "Notebook profissional Intel i7, 16GB RAM, SSD 512GB, tela 15.6\" Full HD",
            "warranty_months": 24,
            "warranty_type": "Garantia estendida incluída",
            "shipping_weight_kg": 3.2,
            "dimensions": "36cm x 24cm x 2cm",
            "common_issues": [
                {
                    "issue": "Demora na entrega (produto importado)",
                    "frequency": "Alta",
                    "resolution": "Comunicação proativa sobre prazo + compensação"
                },
                {
                    "issue": "Teclado com teclas soltas",
                    "frequency": "Baixa",
                    "resolution": "Troca do teclado em assistência técnica"
                }
            ],
            "return_policy_days": 30,
            "supplier": "CompuMax Internacional",
            "manufacturing_origin": "Taiwan",
            "release_date": "2024-08-20"
        },
        
        "PROD-003": {
            "product_id": "PROD-003",
            "product_name": "Fone Bluetooth ABC Premium",
            "category": "Eletrônicos",
            "subcategory": "Áudio",
            "brand": "SoundWave",
            "price": 350.00,
            "original_price": 450.00,
            "discount_percentage": 22,
            "stock_status": "Em estoque",
            "stock_quantity": 120,
            "average_rating": 4.5,
            "total_reviews": 892,
            "description": "Fone Bluetooth com cancelamento de ruído, bateria 30h, resistente à água",
            "warranty_months": 12,
            "warranty_type": "Garantia do fabricante",
            "shipping_weight_kg": 0.3,
            "dimensions": "18cm x 15cm x 8cm (com case)",
            "common_issues": [
                {
                    "issue": "Problema de conexão Bluetooth",
                    "frequency": "Média",
                    "resolution": "Tutorial de reset + troca se persistir"
                },
                {
                    "issue": "Bateria não dura o anunciado",
                    "frequency": "Baixa",
                    "resolution": "Orientação de uso correto + troca"
                }
            ],
            "return_policy_days": 7,
            "supplier": "SoundWave Brasil",
            "manufacturing_origin": "Brasil",
            "release_date": "2024-11-10"
        },
        
        "PROD-004": {
            "product_id": "PROD-004",
            "product_name": "Smart TV 55\" 4K Ultra",
            "category": "Eletrônicos",
            "subcategory": "TVs",
            "brand": "VisionTech",
            "price": 2800.00,
            "original_price": 3500.00,
            "discount_percentage": 20,
            "stock_status": "Em estoque",
            "stock_quantity": 22,
            "average_rating": 4.6,
            "total_reviews": 445,
            "description": "Smart TV 55\" 4K, HDR, Android TV, 3 HDMI, 2 USB",
            "warranty_months": 12,
            "warranty_type": "Garantia do fabricante + suporte técnico",
            "shipping_weight_kg": 18.5,
            "dimensions": "123cm x 71cm x 8cm",
            "common_issues": [
                {
                    "issue": "Tela danificada no transporte",
                    "frequency": "Média",
                    "resolution": "Troca imediata + seguro de transporte"
                },
                {
                    "issue": "Problema com aplicativos travando",
                    "frequency": "Baixa",
                    "resolution": "Atualização de software + suporte técnico"
                },
                {
                    "issue": "Atraso na entrega por tamanho",
                    "frequency": "Alta",
                    "resolution": "Prazo realista + rastreamento prioritário"
                }
            ],
            "return_policy_days": 30,
            "supplier": "VisionTech Indústria",
            "manufacturing_origin": "Brasil",
            "release_date": "2025-03-01"
        },
        
        "PROD-005": {
            "product_id": "PROD-005",
            "product_name": "Mouse Wireless Ergonômico",
            "category": "Informática",
            "subcategory": "Periféricos",
            "brand": "ErgoTech",
            "price": 120.00,
            "original_price": 150.00,
            "discount_percentage": 20,
            "stock_status": "Em estoque",
            "stock_quantity": 250,
            "average_rating": 4.3,
            "total_reviews": 1024,
            "description": "Mouse wireless ergonômico, 6 botões, DPI ajustável, bateria recarregável",
            "warranty_months": 6,
            "warranty_type": "Garantia do fabricante",
            "shipping_weight_kg": 0.2,
            "dimensions": "12cm x 8cm x 5cm",
            "common_issues": [
                {
                    "issue": "Bateria não carrega",
                    "frequency": "Baixa",
                    "resolution": "Troca imediata do produto"
                },
                {
                    "issue": "Conexão USB instável",
                    "frequency": "Média",
                    "resolution": "Envio de novo adaptador USB + troca se necessário"
                }
            ],
            "return_policy_days": 7,
            "supplier": "ErgoTech Supply",
            "manufacturing_origin": "China",
            "release_date": "2024-06-15"
        },
        
        "PROD-006": {
            "product_id": "PROD-006",
            "product_name": "Teclado Mecânico RGB Gamer",
            "category": "Informática",
            "subcategory": "Periféricos",
            "brand": "GamePro",
            "price": 450.00,
            "original_price": 599.00,
            "discount_percentage": 25,
            "stock_status": "Em estoque",
            "stock_quantity": 67,
            "average_rating": 4.8,
            "total_reviews": 723,
            "description": "Teclado mecânico RGB, switches Blue, anti-ghosting, USB-C",
            "warranty_months": 24,
            "warranty_type": "Garantia estendida do fabricante",
            "shipping_weight_kg": 1.1,
            "dimensions": "44cm x 13cm x 4cm",
            "common_issues": [
                {
                    "issue": "Teclas com LED queimado",
                    "frequency": "Baixa",
                    "resolution": "Troca do produto ou reparo em assistência"
                },
                {
                    "issue": "Software de customização não instala",
                    "frequency": "Média",
                    "resolution": "Suporte técnico + link alternativo de download"
                }
            ],
            "return_policy_days": 30,
            "supplier": "GamePro Tech",
            "manufacturing_origin": "China",
            "release_date": "2024-10-05"
        },
        
        "PROD-007": {
            "product_id": "PROD-007",
            "product_name": "Webcam Full HD 1080p",
            "category": "Informática",
            "subcategory": "Periféricos",
            "brand": "StreamCam",
            "price": 280.00,
            "original_price": 320.00,
            "discount_percentage": 13,
            "stock_status": "Em estoque",
            "stock_quantity": 95,
            "average_rating": 4.4,
            "total_reviews": 512,
            "description": "Webcam Full HD 1080p, microfone integrado, foco automático, USB plug-and-play",
            "warranty_months": 12,
            "warranty_type": "Garantia do fabricante",
            "shipping_weight_kg": 0.4,
            "dimensions": "10cm x 7cm x 7cm",
            "common_issues": [
                {
                    "issue": "Imagem desfocada ou borrada",
                    "frequency": "Baixa",
                    "resolution": "Tutorial de ajuste + troca se defeito"
                },
                {
                    "issue": "Não reconhecida pelo computador",
                    "frequency": "Média",
                    "resolution": "Suporte para instalação de drivers + troca"
                }
            ],
            "return_policy_days": 15,
            "supplier": "StreamCam Brasil",
            "manufacturing_origin": "China",
            "release_date": "2025-02-20"
        },
        
        "PROD-008": {
            "product_id": "PROD-008",
            "product_name": "Capa Protetora Premium para iPhone",
            "category": "Acessórios",
            "subcategory": "Capas",
            "brand": "ShieldCase",
            "price": 89.90,
            "original_price": 119.90,
            "discount_percentage": 25,
            "stock_status": "Em estoque",
            "stock_quantity": 340,
            "average_rating": 4.1,
            "total_reviews": 1567,
            "description": "Capa protetora premium, anti-impacto, transparente, borda elevada",
            "warranty_months": 3,
            "warranty_type": "Garantia contra defeitos de fabricação",
            "shipping_weight_kg": 0.05,
            "dimensions": "15cm x 8cm x 1cm",
            "common_issues": [
                {
                    "issue": "Amarelamento rápido da capa",
                    "frequency": "Média",
                    "resolution": "Orientação sobre cuidados + troca se dentro da garantia"
                },
                {
                    "issue": "Não encaixa perfeitamente no modelo",
                    "frequency": "Baixa",
                    "resolution": "Verificação de compatibilidade + troca"
                }
            ],
            "return_policy_days": 7,
            "supplier": "ShieldCase Import",
            "manufacturing_origin": "China",
            "release_date": "2024-12-01"
        },
        
        "PROD-009": {
            "product_id": "PROD-009",
            "product_name": "Carregador Rápido 65W USB-C",
            "category": "Acessórios",
            "subcategory": "Carregadores",
            "brand": "PowerMax",
            "price": 145.00,
            "original_price": 180.00,
            "discount_percentage": 19,
            "stock_status": "Em estoque",
            "stock_quantity": 180,
            "average_rating": 4.6,
            "total_reviews": 892,
            "description": "Carregador rápido 65W, USB-C, compatível com notebooks e smartphones, compacto",
            "warranty_months": 12,
            "warranty_type": "Garantia do fabricante",
            "shipping_weight_kg": 0.25,
            "dimensions": "7cm x 7cm x 3cm",
            "common_issues": [
                {
                    "issue": "Aquecimento excessivo durante uso",
                    "frequency": "Baixa",
                    "resolution": "Orientação sobre uso correto + troca se anormal"
                },
                {
                    "issue": "Não carrega dispositivo específico",
                    "frequency": "Baixa",
                    "resolution": "Verificação de compatibilidade + suporte técnico"
                }
            ],
            "return_policy_days": 15,
            "supplier": "PowerMax Electronics",
            "manufacturing_origin": "China",
            "release_date": "2024-09-10"
        },
        
        "PROD-010": {
            "product_id": "PROD-010",
            "product_name": "SSD Externo 1TB USB 3.2",
            "category": "Armazenamento",
            "subcategory": "SSDs",
            "brand": "DataSpeed",
            "price": 520.00,
            "original_price": 650.00,
            "discount_percentage": 20,
            "stock_status": "Estoque baixo",
            "stock_quantity": 12,
            "average_rating": 4.7,
            "total_reviews": 345,
            "description": "SSD externo 1TB, USB 3.2, velocidade até 1050MB/s, compacto e portátil",
            "warranty_months": 36,
            "warranty_type": "Garantia estendida do fabricante",
            "shipping_weight_kg": 0.15,
            "dimensions": "10cm x 5cm x 1cm",
            "common_issues": [
                {
                    "issue": "Não é reconhecido pelo computador",
                    "frequency": "Baixa",
                    "resolution": "Suporte para formatação + troca se defeito"
                },
                {
                    "issue": "Velocidade abaixo do especificado",
                    "frequency": "Baixa",
                    "resolution": "Verificação de porta USB + orientação técnica"
                }
            ],
            "return_policy_days": 30,
            "supplier": "DataSpeed Technology",
            "manufacturing_origin": "Coreia do Sul",
            "release_date": "2025-01-05"
        }
    }
    
    # Return product info or None if not found
    product = mock_products.get(product_id)
    
    if product:
        return product
    else:
        return None


## Agent 4: Workflow Orchestrator
Agente coordenador responsável por gerenciar o fluxo entre todos os agentes, decidir quais reviews precisam de resposta, determinar quando escalar versus responder automaticamente e garantir SLAs de atendimento.

In [11]:
workflow_orchestrator_prompt = """
You are the Workflow Orchestrator agent (Sales Manager) for the review management system. You coordinate all other agents and make decisions about the review handling workflow.

INPUT FORMAT:
- New review with metadata (JSON)
- System status: queue sizes, available agents, current SLA metrics

YOUR TASKS:
1. Receive new review and trigger review_analyzer agent
2. Based on analysis results, decide workflow path:
   - Path A: Positive/Neutral reviews → no response needed
   - Path B: Negative Low/Medium urgency → Send to response_generator agent ONLY
   - Path C: Negative High urgency → Send to BOTH response_generator agent AND escalation_manager agent
3. Prioritize reviews in queue based on urgency and customer value
4. Monitor SLA compliance (Target: respond to negative reviews within 24h)
5. Generate daily workflow summary

DECISION LOGIC:
- IF sentiment = Positive OR Neutral → Archive only
- IF sentiment = Negative AND urgency = Low/Medium → Generate response
- IF sentiment = Negative AND urgency = High → Generate response + Escalate
- IF legal keywords detected → Always escalate regardless of sentiment

OUTPUT FORMAT (JSON):
{
  "review_id": "string",
  "workflow_path": "Archive|Response_Only|Response_And_Escalate",
  "agents_triggered": ["Review_Analyzer", "Response_Generator", "Escalation_Manager"],
  "priority_level": 1-5,
  "estimated_completion_time": "string (e.g., '2 hours', '24 hours')",
  "sla_status": "Within_SLA|At_Risk|Breached",
  "notes": "string (optional workflow decisions)"
}

Rules:
- Always analyze every review first (trigger Agent 1)
- Never respond to positive reviews automatically
- Prioritize high-urgency reviews in queue
- Track SLA for all negative reviews
- Escalate immediately if legal keywords detected
- Generate response before escalation (so supervisor has draft ready)
- Keep workflow efficient - don't over-escalate
"""

workflow_orchestrator_agent = Agent(
     name="workflow_orchestrator", 
     instructions=workflow_orchestrator_prompt,
     tools=[review_analyzer_tool, response_generator_tool, escalation_manager_tool, get_customer_history, get_product_info],
     handoffs=["review_analyzer", "response_generator", "escalation_manager"],
     handoff_description="Coordinates the review management workflow by triggering appropriate agents based on analysis results.",
     model="gpt-4o-mini")